In [30]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


customer_data = pd.read_csv('Customers.csv')  
transaction_data = pd.read_csv('Transactions.csv')  
product_data = pd.read_csv('Products.csv')  


combined_data = pd.merge(transaction_data, customer_data, on='CustomerID', how='inner')


combined_data = pd.merge(combined_data, product_data, on='ProductID', how='inner')


combined_data.fillna(0, inplace=True)


combined_data['TransactionDate'] = pd.to_datetime(combined_data['TransactionDate'], errors='coerce')  


combined_data['transaction_year'] = combined_data['TransactionDate'].dt.year
combined_data['transaction_month'] = combined_data['TransactionDate'].dt.month
combined_data['transaction_day'] = combined_data['TransactionDate'].dt.day


combined_data.drop(columns=['TransactionDate'], inplace=True)


scaler = StandardScaler()
combined_data[['Quantity', 'TotalValue']] = scaler.fit_transform(
    combined_data[['Quantity', 'TotalValue']]
)


encoded_data = pd.get_dummies(combined_data, columns=['Category', 'Region'], drop_first=True)  


features = encoded_data.drop(columns=['CustomerID', 'CustomerName', 'TransactionID'])  


X = features.select_dtypes(include=['float64', 'int64']).values  

print("Feature Matrix Shape:", X.shape)
print("Sample of Feature Matrix:\n", X[:5])  


knn = NearestNeighbors(n_neighbors=3, metric='cosine')
knn.fit(X)


lookalike_data = []
for customer_index in range(len(X)):
    distances, indices = knn.kneighbors([X[customer_index]]) 
    source_customer = combined_data.iloc[customer_index]['CustomerID']

    for i, idx in enumerate(indices[0]):
        similar_customer = combined_data.iloc[idx]['CustomerID']
        score = 1 - distances[0][i] 
        lookalike_data.append({
            'SourceCustomer': source_customer,
            'SimilarCustomer': similar_customer,
            'SimilarityScore': score
        })


lookalike_df = pd.DataFrame(lookalike_data)


lookalike_df.to_csv('Poornima M_Nerale_Lookalike1.csv', index=False)

print(" Generated!")


Feature Matrix Shape: (1000, 4)
Sample of Feature Matrix:
 [[-1.37548802e+00 -7.89850402e-01  3.00680000e+02  3.00680000e+02]
 [-1.37548802e+00 -7.89850402e-01  3.00680000e+02  3.00680000e+02]
 [-1.37548802e+00 -7.89850402e-01  3.00680000e+02  3.00680000e+02]
 [-4.80570634e-01 -1.79825416e-01  3.00680000e+02  3.00680000e+02]
 [ 4.14346748e-01  4.30199569e-01  3.00680000e+02  3.00680000e+02]]
 Generated!
